## WLAN V11 Sweep

### 0. Constants

In [1]:
PROJECT_NAME = "redfinch"  # decide with db to use
STDF_FILE_NAME = "sweep_with_svc.stdf.gz"  # should be in the same folder with this notebook
TEST_SUITE_NAME = "Main.WLAN.RFU_RX_char.a_gainXrx_11axmcs0_2437_x_x__6dbgstepbw20"

### 1. Upload stdf
Set connection string for uploading records.<br/>
<code>export DB_CONN_STR=mongodb://\<your_conn_str\> </code>


In [3]:
import os
from pymongo import MongoClient
from stdfparser import parse, StdfPub, StdfSubWlan, DBConn

db_conn = DBConn("redfinch",)
stdf_path = os.path.join(os.path.abspath(""), STDF_FILE_NAME)

# parse
if db_conn.get_mir_id(stdf_path) is None:  # bypass stdf already in db
    pub = StdfPub(STDF_FILE_NAME)
    sub = StdfSubWlan(db_conn, stdf_path)
    parse(pub, sub)

parse stdf sweep_with_svc successfully; mir_id=636a07897e438360e9cd27a3


### 2. Fetch data

In [ ]:
import pandas as pd
import numpy as np
from fetch import get_records_v11

df = pd.DataFrame.from_records(get_records_v11(db_conn, TEST_SUITE_NAME))
df.head()

### 3. Plot

In [ ]:
from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

data = {}
for column in df.columns:
    data[column] = df[column].tolist()
    
plot = ggplot(data=data) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="v11",  format=["V11: {} V"], ncol=2)

ggsave(plot)

### SVC Offsets

In [ ]:
from fetch import get_records_svc
svc_dict = get_records_svc(db_conn, text="svc1")
df["svc"] = [svc_dict.get(part_id, 0) for part_id in data["part_id"]]
df["svc_offset"] = df["v11"] - df["svc"]
df["svc_offset_index"] = df["svc_offset"] / 0.015
df = df.round({"svc_offset_index": 0})
df["svc_offset_index"] *= 15
df = df[df["svc_offset_index"] >= 0]
df.head()

In [ ]:
data = {}
for column in df.columns:
    data[column] = df[column].tolist()

ggplot(data=data) \
    + geom_point(mapping = aes(x='pout', y='rssi_min', color='part_id')) \
    + scale_color_discrete() \
    + facet_wrap(facets="svc_offset_index",  format=["SVC Offset: {} mV"], ncol=2)